In [1]:
import csv
from scipy import ndimage
from scipy import linalg
import numpy as np
import nibabel as nib
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import pickle

In [2]:
import random

index = np.array(range(817))
random.seed(40)
random.shuffle(index)

train_index = index[0:572]
val_index = index[572:654]
test_index = index[654:817]

In [3]:
#dataimage_train = [dataimage[j] for j in train_index]
#print(dataimage_train)

In [4]:
lab = np.zeros((817,1)) #first AD:188 then CN:288
lab[0:188, 0] = 1 #AD -- 1; CN -- 0; MCI -- 2
lab[416:817, 0] = 2
for i in range(0, 817):
    print(i, lab[i])

0 [1.]
1 [1.]
2 [1.]
3 [1.]
4 [1.]
5 [1.]
6 [1.]
7 [1.]
8 [1.]
9 [1.]
10 [1.]
11 [1.]
12 [1.]
13 [1.]
14 [1.]
15 [1.]
16 [1.]
17 [1.]
18 [1.]
19 [1.]
20 [1.]
21 [1.]
22 [1.]
23 [1.]
24 [1.]
25 [1.]
26 [1.]
27 [1.]
28 [1.]
29 [1.]
30 [1.]
31 [1.]
32 [1.]
33 [1.]
34 [1.]
35 [1.]
36 [1.]
37 [1.]
38 [1.]
39 [1.]
40 [1.]
41 [1.]
42 [1.]
43 [1.]
44 [1.]
45 [1.]
46 [1.]
47 [1.]
48 [1.]
49 [1.]
50 [1.]
51 [1.]
52 [1.]
53 [1.]
54 [1.]
55 [1.]
56 [1.]
57 [1.]
58 [1.]
59 [1.]
60 [1.]
61 [1.]
62 [1.]
63 [1.]
64 [1.]
65 [1.]
66 [1.]
67 [1.]
68 [1.]
69 [1.]
70 [1.]
71 [1.]
72 [1.]
73 [1.]
74 [1.]
75 [1.]
76 [1.]
77 [1.]
78 [1.]
79 [1.]
80 [1.]
81 [1.]
82 [1.]
83 [1.]
84 [1.]
85 [1.]
86 [1.]
87 [1.]
88 [1.]
89 [1.]
90 [1.]
91 [1.]
92 [1.]
93 [1.]
94 [1.]
95 [1.]
96 [1.]
97 [1.]
98 [1.]
99 [1.]
100 [1.]
101 [1.]
102 [1.]
103 [1.]
104 [1.]
105 [1.]
106 [1.]
107 [1.]
108 [1.]
109 [1.]
110 [1.]
111 [1.]
112 [1.]
113 [1.]
114 [1.]
115 [1.]
116 [1.]
117 [1.]
118 [1.]
119 [1.]
120 [1.]
121 [1.]
122 [1.]
123

In [5]:
mci = 0.
ad = 0.
cn = 0.
total_num = 0.
for i in lab:
    if(i == 1):
        ad += 1
    elif(i == 2):
        mci += 1
    elif(i == 0):
        cn += 1
total_num = mci + ad + cn
print(mci, ad, cn, total_num)

401.0 188.0 228.0 817.0


In [10]:
from torch.utils.data import Dataset, DataLoader

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

class ADNI_Dataset(Dataset):
    
    def __init__(self, ptype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #if(ptype == 'train'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        self.ptype = ptype
        
        self.img = np.load('GRAY_AD_CN_MCI_Jan27.npy') # 416 * ((121 * 121) * 3)
        self.lab = np.zeros((817,1)) #first AD:188 then CN:288
        self.lab[0:188, 0] = 1 #AD -- 1; CN -- 0; MCI -- 2
        #self.lab[416:817, 0] = 2
        self.lab[416:817, 0] = 1
        #for i in range(188):
            #lab[i] = 1
        if(self.ptype == 'train'):
            #count = 0
            self.img = self.img[train_index]
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            for i in range(572):
                for j in range(3):
                    self.img[i][j] = self.img[i][j] + np.random.normal(0, 0.01, (121, 121))
                    print(self.img[i][j].shape)
                    #count += 1
            #print(count)
            self.lab = self.lab[train_index]
        if(self.ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[val_index]
            self.lab = self.lab[val_index]
        if(self.ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[test_index]
            self.lab = self.lab[test_index]
        
        print("great job dude")
        

    def __len__(self):#return int: number of mri
        return len(self.img)

    def __getitem__(self, idx):#input the dataset and desires i-th item
       
        sample = {'image': self.img[idx], 'labels': self.lab[idx,0]} 

        return sample


In [11]:
ADNI_Data_train = ADNI_Dataset('train')

ADNI_Data_val = ADNI_Dataset('val')

ADNI_Data_test = ADNI_Dataset('test')



dataloader_train = DataLoader(ADNI_Data_train, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(ADNI_Data_val, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_test = DataLoader(ADNI_Data_test, batch_size=400,
                        shuffle=True, num_workers=4)

print("finished")

(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)

(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)

In [12]:
import torch.nn as nn
import torch.nn.functional as F

#initial: 
#121 * 121 * 3(feature map number)
#60 * 60 * 16
#30 * 30 * 32
#15 * 15 * 64
#7 * 7 * 128
#3 * 3 * 256

#Linear: 3 * 3 * 256
#3 * 3 * 256(2304)--->1024 --->256 ---> 64 --->3 
class Net_2D(nn.Module):
    def __init__(self):
        super(Net_2D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
        self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv4 = nn.Conv2d(64, 128, 5, padding=2)
        self.conv5 = nn.Conv2d(128, 256, 5, padding=2)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(3 * 3 * 256, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 2)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1, 3 * 3 * 256)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

net_2d = Net_2D().cuda()

print("finished")

finished


In [13]:
import torch.optim as optim
import torch 
Weight = torch.tensor([total_num / (mci + ad), total_num / cn])
criterion = nn.CrossEntropyLoss(weight = Weight.cuda())
optimizer = optim.SGD(net_2d.parameters(), lr=0.0001, momentum=0.9)#future 0.0001

print("finished")

finished


In [14]:
import numpy as np
import torch

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

loss_val_min = 1000000
threshold = 10000

epoch = 10000
min_total_loss = 1000000


for i in range(epoch):
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched[images].shape)
        #print(i_batch)
        #print(i_batch,
        #      sample_batched['patient_id'])
        
        labels = sample_batched['labels'].cuda()
        #labels_arr = np.asarray(sample_batched['labels'])
        #print(labels_arr.shape)
        running_loss = 0.0
       
    # get the inputs
        
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        images = sample_batched['image'].float().cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()



        # zero the parameter gradients
        optimizer.zero_grad()
        
        #before actual training
        net_2d.train()

        # forward + backward + optimize
        outputs = net_2d(images)
        #print(outputs)
        #print(labels.long())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))

    print('$$Finished Training This Epoch')
    
    #validation for the epoch
    net_2d.eval()
    total_val_loss = 0
    print('Start a new val_epoch')
    for i_batch, sample_batched in enumerate(dataloader_val):
        #print(i_batch)
              #sample_batched['patient_id'])

        running_loss = 0.0
        
        labels = sample_batched['labels'].cuda()
        images = sample_batched['image'].float().cuda()
        # get the inputs
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()
        
        outputs = net_2d(images)
        loss = criterion(outputs, labels.long())
        print('Validation')
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))
        #print(loss.item())
        #print(type(loss.item()))
        print('\n')
        
        total_val_loss = total_val_loss + loss.cpu().data.numpy()
        
    if(total_val_loss >= loss_val_min):
        if(total_val_loss < 0.5):
            count += 1
    else:
        loss_val_min = total_val_loss
        count = 0
        torch.save(net_2d, 'best_model_Jan30_2d_3types_nonoverfit_2-2.pkl')
    #if(total_val_loss < min_total_loss):
        #min_total_loss = total_val_loss
        #save best model
        #torch.save(net_2d, 'best_model_Jan27_2d_3types.pkl')
    if(count > threshold):
        break
    

print("we're done training")

[1] loss: 0.746
[2] loss: 0.744
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.748




/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net_2D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[1] loss: 0.745
[2] loss: 0.746
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.748


[1] loss: 0.746
[2] loss: 0.743
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.748


[1] loss: 0.744
[2] loss: 0.747
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.747


[1] loss: 0.745
[2] loss: 0.743
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.747


[1] loss: 0.743
[2] loss: 0.745
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.746


[1] loss: 0.743
[2] loss: 0.743
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.746


[1] loss: 0.742
[2] loss: 0.744
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.745


[1] loss: 0.741
[2] loss: 0.745
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.744


[1] loss: 0.740
[2] loss: 0.744
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.695
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.694


[1] loss: 0.694
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.694


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.693


[1] loss: 0.693
[2] loss: 0.692
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.692


[1] loss: 0.692
[2] loss: 0.692
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.691


[1] loss: 0.691
[2] loss: 0.691
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.691


[1] loss: 0.691
[2] loss: 0.690
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.690


[1] loss: 0.690
[2] loss: 0.690
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.689


[1] loss: 0.690
[2] loss: 0.689
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.651
[2] loss: 0.649
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.648


[1] loss: 0.650
[2] loss: 0.650
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.647


[1] loss: 0.650
[2] loss: 0.648
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.647


[1] loss: 0.649
[2] loss: 0.649
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.646


[1] loss: 0.651
[2] loss: 0.643
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.645


[1] loss: 0.648
[2] loss: 0.648
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.645


[1] loss: 0.647
[2] loss: 0.649
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.644


[1] loss: 0.647
[2] loss: 0.648
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.644


[1] loss: 0.646
[2] loss: 0.647
$$Finished Training This Epoch
Start a new val_epoch
Val

...

[1] loss: 0.391
[2] loss: 0.393
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.391
[2] loss: 0.393
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.370
[2] loss: 0.444
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.391
[2] loss: 0.393
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.396
[2] loss: 0.381
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.401
[2] loss: 0.369
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.399
[2] loss: 0.375
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.377
[2] loss: 0.425
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.396
[2] loss: 0.381
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.383
[2] loss: 0.412
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.407
[2] loss: 0.357
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.377
[2] loss: 0.425
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.372
[2] loss: 0.438
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.401
[2] loss: 0.369
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.383
[2] loss: 0.412
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.372
[2] loss: 0.438
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.401
[2] loss: 0.369
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.399
[2] loss: 0.375
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.380
[2] loss: 0.418
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.391
[2] loss: 0.393
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.396
[2] loss: 0.381
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.385
[2] loss: 0.406
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.396
[2] loss: 0.381
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.401
[2] loss: 0.369
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.391
[2] loss: 0.393
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.375
[2] loss: 0.431
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.388
[2] loss: 0.399
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.407
[2] loss: 0.357
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.385
[2] loss: 0.406
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.385
[2] loss: 0.406
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.404
[2] loss: 0.363
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.391
[2] loss: 0.393
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.393
[2] loss: 0.387
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.401
[2] loss: 0.369
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.401
[2] loss: 0.369
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.383
[2] loss: 0.412
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.388
[2] loss: 0.399
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.396
[2] loss: 0.381
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.388
[2] loss: 0.399
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.409
[2] loss: 0.351
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.385
[2] loss: 0.406
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.365
[2] loss: 0.458
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.383
[2] loss: 0.412
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.375
[2] loss: 0.431
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.351


[1] loss: 0.407
[2] loss: 0.357
$$Finished Training This Epoch
Start a new val_epoch
Val

In [15]:
import torch
#test after finish all epochs

#load the best_model back 
final_net = torch.load('best_model_Jan30_2d_3types_nonoverfit_2-2.pkl')
final_net.eval()
correct_sum = 0.
total_sum = 0.
for i_batch, sample_batched in enumerate(dataloader_test):
    #print(i_batch)

    running_loss = 0.0
    labels = sample_batched['labels'].cuda()
    # get the inputs
    # labels_arr = np.asarray(sample_batched['labels'])
    # labels_arr = labels_arr.astype(np.int)
    # labels = ToTensor(labels_arr).cuda()
    images = sample_batched['image'].float().cuda()
    # images_arr = np.asarray(sample_batched['image'])
    # images_arr = images_arr.astype(np.float)
    # images = ToTensor(images_arr).cuda()

    outputs = final_net(images)#n * 3
    results_type = np.argmax(outputs.cpu().data.numpy(), axis=1)
    results_type = ToTensor(results_type)
    #print(outputs)

    #calculate accuracy
    labels = labels.cpu()
    print("predicted:\n", results_type, "\nactual:\n", labels)

for i in range(len(results_type)):
    if(labels[i].float() == results_type[i].float()):
        correct_sum += 1
    total_sum += 1


print("correct_sum_final", correct_sum)
print(total_sum)


predicted:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.]) 
actual:
 tensor([1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1.,
        0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
     

In [ ]:
from torch.utils.data import Dataset, DataLoader

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

class ADNI_Dataset_New(Dataset):
    
    def __init__(self, ptype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #if(ptype == 'train'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        self.ptype = ptype
        
        self.img = np.load('GRAY_AD_CN_MCI_Jan27.npy') # 416 * ((121 * 121) * 3)
        self.lab = np.zeros((817,1)) #first AD:188 then CN:288
        self.lab[0:188, 0] = 0 #AD -- 0; CN -- NO; MCI -- 1
        #self.lab[416:817, 0] = 2
        self.lab[416:817, 0] = 1
        #for i in range(188):
            #lab[i] = 1
        if(self.ptype == 'train'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[train_index]
            self.lab = self.lab[train_index]
            train_index_new = np.where(self.lab != 0)
            self.lab_new = self.lab[train_index_new]
            self.img_new = self.img[train_index_new]
            
        if(self.ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[val_index]
            self.lab = self.lab[val_index]
            val_index_new = np.where(self.lab != 0)
            self.lab_new = self.lab[val_index_new]
            self.img_new = self.img[val_index_new]
        if(self.ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[test_index]
            self.lab = self.lab[test_index]
            test_index_new = np.where(self.lab != 0)
            self.lab_new = self.lab[test_index_new]
            self.img_new = self.img[test_index_new]
        
        print("great job dude")
        

    def __len__(self):#return int: number of mri
        return len(self.img)

    def __getitem__(self, idx):#input the dataset and desires i-th item
       
        sample = {'image': self.img[idx], 'labels': self.lab[idx,0]} 

        return sample


In [ ]:
ADNI_Data_train = ADNI_Dataset_New('train')

ADNI_Data_val = ADNI_Dataset_New('val')

ADNI_Data_test = ADNI_Dataset_New('test')



dataloader_train = DataLoader(ADNI_Data_train, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(ADNI_Data_val, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_test = DataLoader(ADNI_Data_test, batch_size=400,
                        shuffle=True, num_workers=4)

print("finished")

In [ ]:
import torch.optim as optim
import torch 
Weight = torch.tensor([total_num / ad, total_num / mci])
criterion = nn.CrossEntropyLoss(weight = Weight.cuda())
optimizer = optim.SGD(net_2d.parameters(), lr=0.0005, momentum=0.9)

print("finished")

In [ ]:
import numpy as np
import torch

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

loss_val_min = 1000000
threshold = 10000

epoch = 10000
min_total_loss = 1000000


for i in range(epoch):
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched[images].shape)
        #print(i_batch)
        #print(i_batch,
        #      sample_batched['patient_id'])
        
        labels = sample_batched['labels'].cuda()
        #labels_arr = np.asarray(sample_batched['labels'])
        #print(labels_arr.shape)
        running_loss = 0.0
       
    # get the inputs
        
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        images = sample_batched['image'].float().cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()



        # zero the parameter gradients
        optimizer.zero_grad()
        
        #before actual training
        net_2d.train()

        # forward + backward + optimize
        outputs = net_2d(images)
        #print(outputs)
        #print(labels.long())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))

    print('$$Finished Training This Epoch')
    
    #validation for the epoch
    net_2d.eval()
    total_val_loss = 0
    print('Start a new val_epoch')
    for i_batch, sample_batched in enumerate(dataloader_val):
        #print(i_batch)
              #sample_batched['patient_id'])

        running_loss = 0.0
        
        labels = sample_batched['labels'].cuda()
        images = sample_batched['image'].float().cuda()
        # get the inputs
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()
        
        outputs = net_2d(images)
        loss = criterion(outputs, labels.long())
        print('Validation')
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))
        #print(loss.item())
        #print(type(loss.item()))
        print('\n')
        
        total_val_loss = total_val_loss + loss.cpu().data.numpy()
        
    if(total_val_loss >= loss_val_min):
        if(total_val_loss < 0.5):
            count += 1
    else:
        loss_val_min = total_val_loss
        count = 0
        torch.save(net_2d, 'best_model_Jan30_2d_3types_nonoverfit_2-2_AD_MCI.pkl')
    #if(total_val_loss < min_total_loss):
        #min_total_loss = total_val_loss
        #save best model
        #torch.save(net_2d, 'best_model_Jan27_2d_3types.pkl')
    if(count > threshold):
        break
    
print("we're done training")

In [ ]:
import torch
#test after finish all epochs

#load the best_model back 
final_net = torch.load('best_model_Jan30_2d_3types_nonoverfit_2-2_AD_MCI.pkl')
final_net.eval()
correct_sum = 0.
total_sum = 0.
for i_batch, sample_batched in enumerate(dataloader_test):
    #print(i_batch)

    running_loss = 0.0
    labels = sample_batched['labels'].cuda()
    # get the inputs
    # labels_arr = np.asarray(sample_batched['labels'])
    # labels_arr = labels_arr.astype(np.int)
    # labels = ToTensor(labels_arr).cuda()
    images = sample_batched['image'].float().cuda()
    # images_arr = np.asarray(sample_batched['image'])
    # images_arr = images_arr.astype(np.float)
    # images = ToTensor(images_arr).cuda()

    outputs = final_net(images)#n * 3
    results_type = np.argmax(outputs.cpu().data.numpy(), axis=1)
    results_type = ToTensor(results_type)
    #print(outputs)

    #calculate accuracy
    labels = labels.cpu()
    print("predicted:\n", results_type, "\nactual:\n", labels)

for i in range(len(results_type)):
    if(labels[i].float() == results_type[i].float()):
        correct_sum += 1
    total_sum += 1


print("correct_sum_final", correct_sum)
print(total_sum)
